# Creating voting affinity scores for the 118th Congress

In [1]:
import pandas as pd
import numpy as np 

In [2]:
votes = pd.read_csv('https://voteview.com/static/data/out/votes/HS118_votes.csv')
votes = votes[['chamber', 'rollnumber', 'icpsr', 'cast_code']]
house_votes = votes.query("chamber=='House'")
senate_votes = votes.query("chamber=='Senate'")

In [3]:
paired_votes = pd.merge(senate_votes, senate_votes, on = ['chamber', 'rollnumber'], how = 'inner', validate = 'm:m')

In [5]:
paired_votes = paired_votes.rename({'icpsr_x': 'icpsr', 
                                    'icpsr_y': 'member_compare', 
                                   'cast_code_x': 'vote', 
                                   'cast_code_y': 'compare_vote'}, axis = 1)

In [7]:
paired_votes = paired_votes.query("icpsr != member_compare")
paired_votes

,chamber,rollnumber,icpsr,vote,member_compare,compare_vote
1,Senate,1,14226,1,14435,1
2,Senate,1,14226,1,14858,1
3,Senate,1,14226,1,14871,1
4,Senate,1,14226,1,14921,1
5,Senate,1,14226,1,15015,1
...,...,...,...,...,...,...
2529198,Senate,253,91300,1,42303,6
2529199,Senate,253,91300,1,42304,6
2529200,Senate,253,91300,1,42305,1
2529201,Senate,253,91300,1,49308,1


In [9]:
#paired_votes['equal_vote'] = paired_votes['vote'] == paired_votes['compare_vote']
paired_votes = paired_votes.assign(equal_vote = paired_votes['vote'] == paired_votes['compare_vote'])
paired_votes

,chamber,rollnumber,icpsr,vote,member_compare,compare_vote,equal_vote
1,Senate,1,14226,1,14435,1,True
2,Senate,1,14226,1,14858,1,True
3,Senate,1,14226,1,14871,1,True
4,Senate,1,14226,1,14921,1,True
5,Senate,1,14226,1,15015,1,True
...,...,...,...,...,...,...,...
2529198,Senate,253,91300,1,42303,6,False
2529199,Senate,253,91300,1,42304,6,False
2529200,Senate,253,91300,1,42305,1,True
2529201,Senate,253,91300,1,49308,1,True


In [10]:
paired_votes = paired_votes[['icpsr', 'member_compare', 'equal_vote']]

In [24]:
affinity = paired_votes.groupby(['icpsr', 'member_compare']).agg({'equal_vote': 'mean'}).reset_index()
affinity

,icpsr,member_compare,equal_vote
0,14226,14435,0.308300
1,14226,14858,0.379447
2,14226,14871,0.355731
3,14226,14921,0.766798
4,14226,15015,0.359684
...,...,...,...
10093,91300,42304,0.177866
10094,91300,42305,1.000000
10095,91300,49300,0.485597
10096,91300,49308,0.913043


In [26]:
affinity['affinity_score'] = round(affinity['equal_vote']*100, 2).astype(str)+"%"
affinity

,icpsr,member_compare,equal_vote,affinity_score
0,14226,14435,0.308300,30.83%
1,14226,14858,0.379447,37.94%
2,14226,14871,0.355731,35.57%
3,14226,14921,0.766798,76.68%
4,14226,15015,0.359684,35.97%
...,...,...,...,...
10093,91300,42304,0.177866,17.79%
10094,91300,42305,1.000000,100.0%
10095,91300,49300,0.485597,48.56%
10096,91300,49308,0.913043,91.3%


In [27]:
affinity = affinity.drop('equal_vote', axis = 1)
affinity

,icpsr,member_compare,affinity_score
0,14226,14435,30.83%
1,14226,14858,37.94%
2,14226,14871,35.57%
3,14226,14921,76.68%
4,14226,15015,35.97%
...,...,...,...
10093,91300,42304,17.79%
10094,91300,42305,100.0%
10095,91300,49300,48.56%
10096,91300,49308,91.3%


In [28]:
ideology = pd.read_csv('https://voteview.com/static/data/out/members/HS118_members.csv')

In [29]:
ideo_senate = ideology.query("chamber == 'Senate'")
ideo_senate = ideo_senate[['icpsr', 'state_abbrev', 'party_code', 'bioname']]

In [30]:
ideo_senate

,icpsr,state_abbrev,party_code,bioname
441,42102,AL,200,"TUBERVILLE, Thomas Hawley (Tommy)"
442,42300,AL,200,"BRITT, Katie Elizabeth"
443,40300,AK,200,"MURKOWSKI, Lisa"
444,41500,AK,200,"SULLIVAN, Daniel Scott"
445,41905,AZ,100,"KELLY, Mark Edward"
...,...,...,...,...
537,40915,WV,100,"MANCHIN, Joe, III"
538,29940,WI,100,"BALDWIN, Tammy"
539,41111,WI,200,"JOHNSON, Ron"
540,20953,WY,200,"LUMMIS, Cynthia M."


In [32]:
affinity = pd.merge(affinity, ideo_senate, on = 'icpsr', how = 'outer', validate = "m:1")
affinity

,icpsr,member_compare,affinity_score,state_abbrev,party_code,bioname
0,14226,14435,30.83%,IA,200,"GRASSLEY, Charles Ernest"
1,14226,14858,37.94%,IA,200,"GRASSLEY, Charles Ernest"
2,14226,14871,35.57%,IA,200,"GRASSLEY, Charles Ernest"
3,14226,14921,76.68%,IA,200,"GRASSLEY, Charles Ernest"
4,14226,15015,35.97%,IA,200,"GRASSLEY, Charles Ernest"
...,...,...,...,...,...,...
10093,91300,42304,17.79%,AZ,328,"SINEMA, Kyrsten"
10094,91300,42305,100.0%,AZ,328,"SINEMA, Kyrsten"
10095,91300,49300,48.56%,AZ,328,"SINEMA, Kyrsten"
10096,91300,49308,91.3%,AZ,328,"SINEMA, Kyrsten"


In [34]:
affinity = pd.merge(affinity, ideo_senate, left_on = 'member_compare', right_on = 'icpsr', how = 'outer', validate = "m:1")
affinity

,icpsr_x,member_compare,affinity_score,state_abbrev_x,party_code_x,bioname_x,icpsr_y,state_abbrev_y,party_code_y,bioname_y
0,14226,14435,30.83%,IA,200,"GRASSLEY, Charles Ernest",14435,MA,100,"MARKEY, Edward John"
1,14858,14435,90.91%,NY,100,"SCHUMER, Charles Ellis (Chuck)",14435,MA,100,"MARKEY, Edward John"
2,14871,14435,93.28%,OR,100,"WYDEN, Ronald Lee",14435,MA,100,"MARKEY, Edward John"
3,14921,14435,24.11%,KY,200,"McCONNELL, Addison Mitchell (Mitch)",14435,MA,100,"MARKEY, Edward John"
4,15015,14435,92.09%,DE,100,"CARPER, Thomas Richard",14435,MA,100,"MARKEY, Edward John"
...,...,...,...,...,...,...,...,...,...,...
10093,42305,14226,33.33%,CA,100,"BUTLER, Laphonza Romanique",14226,IA,200,"GRASSLEY, Charles Ernest"
10094,49300,14226,13.99%,CA,100,"FEINSTEIN, Dianne",14226,IA,200,"GRASSLEY, Charles Ernest"
10095,49308,14226,36.36%,WA,100,"MURRAY, Patty",14226,IA,200,"GRASSLEY, Charles Ernest"
10096,49703,14226,60.08%,ME,200,"COLLINS, Susan Margaret",14226,IA,200,"GRASSLEY, Charles Ernest"
